***GENERATED CODE FOR wfanoshowcanceldistanceinfoxgbd PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import warnings
warnings.filterwarnings('ignore')


class RDBMSConnector:

    def fetch(spark, config):
        drivers = {"mssql": "com.microsoft.sqlserver.jdbc.SQLServerDriver"}
        return spark.read.format("jdbc") \
            .option("url", f"jdbc:sqlserver://{eval(config)['host']}:{eval(config)['port']};databaseName={eval(config)['database']}") \
            .option(eval(config)['qtype'], eval(config)['query']) \
            .option("user", eval(config)['user']) \
            .option("password", eval(config)['password']) \
            .option("driver",  "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
            .load()

    def put(df, spark, config):
        drivers = {"mssql": "com.microsoft.sqlserver.jdbc.SQLServerDriver"}
        # Write modes: overwrite, append
        df.write.mode(eval(config)['writemode'])\
            .format('jdbc') \
            .option("url", f"jdbc:{eval(config)['dbtype']}://{eval(config)['host']}:{eval(config)['port']};databaseName={eval(config)['database']}") \
            .option("dbtable", eval(config)['table']) \
            .option("user", eval(config)['user']) \
            .option("password", eval(config)['password']) \
            .option("driver",  "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
            .save()


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
from pyspark.sql.functions import dayofmonth, month, year, col
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def replaceByMean(self, feature, df, mean_=-1):

        meanValue = df.select(mean(col(feature.name)).alias(
            'mean')).collect()[0]["mean"]
        df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                                         meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        maxValue = df.select(max(col(feature.name)).alias('max')).collect()[
            0]["max"]
        df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        minValue = df.select(min(col(feature.name)).alias('min')).collect()[
            0]["min"]
        df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        stddevValue = df.select(stddev(col(feature.name)).alias(
            'stddev')).collect()[0]["stddev"]
        df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        fillValue = df.where(col(feature.name).isNotNull()
                             ).head(1)[0][feature.name]
        df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


def ExtractDateTransform(df, params, transformationData={}):
    transform_params = params
    dfReturn = df
    feature = transform_params['feature']
    dfReturn = dfReturn.fillna({feature: ''})
    dfReturn = dfReturn.withColumn(
        feature+'dayofmonth', dayofmonth(col(feature)))
    dfReturn = dfReturn.withColumn(feature+'month', month(col(feature)))
    dfReturn = dfReturn.withColumn(feature+'year', year(col(feature)))
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'policynumber', 'transformation_label': 'String Indexer'}], 'feature': 'policynumber', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '500', 'mean': '6896866993.08', 'stddev': '27954276235.5', 'min': '', 'max': '21551825', 'missing': '0', 'distinct': '402'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'policynumber'}, {'feature_label': 'policynumber', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('policynumber')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'AppointmentID', 'transformation_label': 'String Indexer'}], 'feature': 'AppointmentID', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': '0004A03B-ECD3-40EF-A3D9-AA1875A6752D', 'max': 'FFE29399-288E-4F85-B66D-3D7227A0AED9', 'missing': '0', 'distinct': '500'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'AppointmentID'}, {'feature_label': 'AppointmentID', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('AppointmentID')
        transformationDF = ExtractDateTransform(transformationDF, {'transformationsData': [{'feature_label': 'AppointmentDate', 'transformation_label': 'Extract Date'}], 'feature': 'AppointmentDate', 'type': 'date', 'selected': 'True', 'replaceby': 'random', 'stats': {
            'count': '', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '0'}, 'transformation': [{'transformation': 'Extract Date', 'selectedAsDefault': 1}], 'generated': 'False', 'updatedLabel': 'AppointmentDate'}, {'feature_label': 'AppointmentDate', 'transformation_label': 'Extract Date'})
        transformationDF = transformationDF.drop('AppointmentDate')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'PatientID', 'transformation_label': 'String Indexer'}], 'feature': 'PatientID', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': '017E3AD3-7931-499D-8CEB-C7F5FCAF574C', 'max': 'FF35B960-B0D8-49A2-A9B0-3190583199EA', 'missing': '0', 'distinct': '411'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'PatientID'}, {'feature_label': 'PatientID', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('PatientID')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'PatientNumber', 'transformation_label': 'String Indexer'}], 'feature': 'PatientNumber', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '1210517.23', 'stddev': '849566.96', 'min': '100530', 'max': '99834', 'missing': '0', 'distinct': '411'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'PatientNumber'}, {'feature_label': 'PatientNumber', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('PatientNumber')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Gender', 'transformation_label': 'String Indexer'}], 'feature': 'Gender', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'F', 'max': 'M', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Gender'}, {'feature_label': 'Gender', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Gender')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'PatientCity', 'transformation_label': 'String Indexer'}], 'feature': 'PatientCity', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Atlanta', 'max': 'Wheeling', 'missing': '0', 'distinct': '37'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'PatientCity'}, {'feature_label': 'PatientCity', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('PatientCity')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'PatientZipcode', 'transformation_label': 'String Indexer'}], 'feature': 'PatientZipcode', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '43400.92', 'stddev': '16941.32', 'min': '08807', 'max': '95899', 'missing': '0', 'distinct': '46'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'PatientZipcode'}, {'feature_label': 'PatientZipcode', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('PatientZipcode')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'PatientEthnicityID', 'transformation_label': 'String Indexer'}], 'feature': 'PatientEthnicityID', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': '1bc90f3d-2442-410d-8814-ccc7e81da7a1', 'max': 'cbd0af81-70ad-4e34-afa6-0a5f10ac035c', 'missing': '0', 'distinct': '7'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'PatientEthnicityID'}, {'feature_label': 'PatientEthnicityID', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('PatientEthnicityID')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'PatientRace', 'transformation_label': 'String Indexer'}], 'feature': 'PatientRace', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Asian', 'max': 'White', 'missing': '0', 'distinct': '6'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'PatientRace'}, {'feature_label': 'PatientRace', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('PatientRace')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'PatientType', 'transformation_label': 'String Indexer'}], 'feature': 'PatientType', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'EstPt', 'max': 'NewPt', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'PatientType'}, {'feature_label': 'PatientType', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('PatientType')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'BinsAppointmentDuration', 'transformation_label': 'String Indexer'}], 'feature': 'BinsAppointmentDuration', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': '1-3 hrs', 'max': 'Lt 10 Mins', 'missing': '0', 'distinct': '4'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'BinsAppointmentDuration'}, {'feature_label': 'BinsAppointmentDuration', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('BinsAppointmentDuration')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'TimeOfAppointment', 'transformation_label': 'String Indexer'}], 'feature': 'TimeOfAppointment', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'AM', 'max': 'PM', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'TimeOfAppointment'}, {'feature_label': 'TimeOfAppointment', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('TimeOfAppointment')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'BinsTimeOfAppointment', 'transformation_label': 'String Indexer'}], 'feature': 'BinsTimeOfAppointment', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Early Morning', 'max': 'Post Lunch/Evening', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'BinsTimeOfAppointment'}, {'feature_label': 'BinsTimeOfAppointment', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('BinsTimeOfAppointment')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'AppointmentType', 'transformation_label': 'String Indexer'}], 'feature': 'AppointmentType', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': '1 - Cataract Consult', 'max': 'Z - YAG', 'missing': '0', 'distinct': '73'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'AppointmentType'}, {'feature_label': 'AppointmentType', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('AppointmentType')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'PracticeName', 'transformation_label': 'String Indexer'}], 'feature': 'PracticeName', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'PracticeName1', 'max': 'PracticeName2', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'PracticeName'}, {'feature_label': 'PracticeName', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('PracticeName')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'FacilityCategory', 'transformation_label': 'String Indexer'}], 'feature': 'FacilityCategory', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Location107', 'max': 'Location96', 'missing': '0', 'distinct': '39'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'FacilityCategory'}, {'feature_label': 'FacilityCategory', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('FacilityCategory')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'FacilityCityName', 'transformation_label': 'String Indexer'}], 'feature': 'FacilityCityName', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Batavia', 'max': 'Troy', 'missing': '0', 'distinct': '19'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'FacilityCityName'}, {'feature_label': 'FacilityCityName', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('FacilityCityName')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'FacilityState', 'transformation_label': 'String Indexer'}], 'feature': 'FacilityState', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'IN', 'max': 'OH', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'FacilityState'}, {'feature_label': 'FacilityState', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('FacilityState')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'FacilityZip', 'transformation_label': 'String Indexer'}], 'feature': 'FacilityZip', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '449621676.45', 'stddev': '11066586.44', 'min': '410173415', 'max': '470251055', 'missing': '0', 'distinct': '33'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'FacilityZip'}, {'feature_label': 'FacilityZip', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('FacilityZip')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'FacilityGroup1', 'transformation_label': 'String Indexer'}], 'feature': 'FacilityGroup1', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'ASC', 'max': 'VEI Office', 'missing': '0', 'distinct': '11'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'FacilityGroup1'}, {'feature_label': 'FacilityGroup1', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('FacilityGroup1')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'FacilityGroup2', 'transformation_label': 'String Indexer'}], 'feature': 'FacilityGroup2', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Indiana', 'max': 'Ohio', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'FacilityGroup2'}, {'feature_label': 'FacilityGroup2', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('FacilityGroup2')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'FacilityAddressLine1', 'transformation_label': 'String Indexer'}], 'feature': 'FacilityAddressLine1', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': '10615 Montgomery Rd., Suite 202', 'max': '89 Sylvania Drive', 'missing': '0', 'distinct': '36'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'FacilityAddressLine1'}, {'feature_label': 'FacilityAddressLine1', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('FacilityAddressLine1')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'ProviderName', 'transformation_label': 'String Indexer'}], 'feature': 'ProviderName', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Provider10749', 'max': 'Provider9994', 'missing': '0', 'distinct': '117'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'ProviderName'}, {'feature_label': 'ProviderName', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('ProviderName')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Speciality', 'transformation_label': 'String Indexer'}], 'feature': 'Speciality', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'SpecialityCategory-11_Demo3', 'max': 'SpecialityCategory6465_Demo3', 'missing': '0', 'distinct': '11'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Speciality'}, {'feature_label': 'Speciality', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Speciality')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'ProviderTaxonomyName', 'transformation_label': 'String Indexer'}], 'feature': 'ProviderTaxonomyName', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Allopathic & Osteopathic Physicians : Ophthalmology', 'max': 'Unspecified', 'missing': '0', 'distinct': '7'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'ProviderTaxonomyName'}, {'feature_label': 'ProviderTaxonomyName', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('ProviderTaxonomyName')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'PayerCategory', 'transformation_label': 'String Indexer'}], 'feature': 'PayerCategory', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Aetna Commercial', 'max': 'United Commercial', 'missing': '0', 'distinct': '16'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'PayerCategory'}, {'feature_label': 'PayerCategory', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('PayerCategory')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'PayerGroup', 'transformation_label': 'String Indexer'}], 'feature': 'PayerGroup', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Aetna Commercial', 'max': 'United Commercial', 'missing': '0', 'distinct': '16'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'PayerGroup'}, {'feature_label': 'PayerGroup', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('PayerGroup')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'InsurenceCategory', 'transformation_label': 'String Indexer'}], 'feature': 'InsurenceCategory', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'All Commercial', 'max': 'Others', 'missing': '0', 'distinct': '4'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'InsurenceCategory'}, {'feature_label': 'InsurenceCategory', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('InsurenceCategory')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'prevappttype', 'transformation_label': 'String Indexer'}], 'feature': 'prevappttype', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': '1 - Cataract Consult', 'max': 'Z - Surgery (Plastics)', 'missing': '0', 'distinct': '56'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'prevappttype'}, {'feature_label': 'prevappttype', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('prevappttype')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Age_Bins', 'transformation_label': 'String Indexer'}], 'feature': 'Age_Bins', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': '51-70', 'max': '<=50', 'missing': '0', 'distinct': '4'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Age_Bins'}, {'feature_label': 'Age_Bins', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Age_Bins')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Bins_apptrank', 'transformation_label': 'String Indexer'}], 'feature': 'Bins_apptrank', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '0.0', 'stddev': '0.0', 'min': '0', 'max': '>10', 'missing': '0', 'distinct': '4'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Bins_apptrank'}, {'feature_label': 'Bins_apptrank', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Bins_apptrank')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

***READING DATAFRAME***

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

#%run wfanoshowcanceldistanceinfoxgbdHooks.ipynb
try:
	#sourcePreExecutionHook()

	sqlserver = RDBMSConnector.fetch(spark, "{'host': 'wsc-sqlserver1.database.windows.net', 'port': '1433', 'password': 'G5gTW5rnKVeAZWXm', 'database': 'WorkflowAnalyticsDev', 'optionalDB': 'WorkflowAnalyticsDev', 'user': 'Numtra', 'qtype': 'query', 'query': "select\r\ndistinct (policynumber),\r\nA.AppointmentID,\r\nA.AppointmentDate,\r\nA.PatientID,\r\nPatientNumber,\r\nCASE WHEN p.Gender IS NOT NULL THEN p.Gender ELSE 'NOT SPECIFIED' END as Gender,\r\nFLOOR( DATEDIFF(DAY, P.DateOfBirth, a.AppointmentDate) / 365) AS PatientAge,\r\nCASE WHEN P.City IS NOT NULL THEN P.City ELSE 'NOT SPECIFIED' END as PatientCity,\r\nCASE WHEN P.Zip IS NOT NULL THEN SUBSTRING(P.Zip,1,5) ELSE 'NOT SPECIFIED' END as PatientZipcode,\r\nCASE WHEN P.EthnicityID IS NOT NULL THEN P.EthnicityID ELSE 'NOT SPECIFIED' END as PatientEthnicityID,\r\nCASE WHEN R.Race IS NOT NULL THEN R.Race ELSE 'NOT SPECIFIED' END as PatientRace, round(APS.Distance,0) as Distance,\r\nZC.population, ZC.mean_income,\r\nCASE WHEN AT.AppointmentTypeCategory = '1 - New' THEN 'NewPt' ELSE 'EstPt' END as PatientType ,\r\nDATEDIFF(day, A.CreatedDate, A.AppointmentDate) as days_AppDate_AppCreatDate,\r\nA.CreatedBy,\r\nCASE WHEN A.AppointmentDuration> 0 and A.AppointmentDuration Between 1 and 10 THEN 'Lt 10 Mins'\r\nWHEN A.AppointmentDuration> 10 and A.AppointmentDuration <= 30 THEN '10-30 Mins'\r\nWHEN A.AppointmentDuration> 30 and A.AppointmentDuration <= 60 THEN '30-60 Mins'\r\nWHEN A.AppointmentDuration> 60 and A.AppointmentDuration <= 180 THEN '1-3 hrs'\r\nWHEN A.AppointmentDuration> 180 THEN 'Gt 3 hrs'\r\nELSE 'NoDuration' END AS BinsAppointmentDuration,\r\n\r\nCASE WHEN Format(A.AppointmentStart, '0000') Between 0000 and 1200 THEN 'AM' ELSE 'PM' END AS TimeOfAppointment,\r\n\r\nCASE WHEN Format(A.AppointmentStart, '0000') Between 0000 and 0800 THEN 'Early Morning'\r\nWHEN Format(A.AppointmentStart, '0000') Between 0801 and 1200 THEN 'Morning'\r\nWHEN Format(A.AppointmentStart, '0000') Between 1201 and 2300 THEN 'Post Lunch/Evening'\r\nELSE 'Late Night' END AS BinsTimeOfAppointment,\r\n\r\nDATEPART(WEEKDAY, A.AppointmentDate) as DayoftheWeek,\r\nDATEPART(MONTH, A.AppointmentDate) as MonthoftheYear,\r\nAT.AppointmentTypeCategory as AppointmentType,\r\nA.EligibilityFlag,\r\nA.ConfirmationFlag,\r\nPRC.PracticeName,\r\nF.FacilityCategory, LEFT(F.City, LEN(F.City) - PATINDEX ('%[0-9][^0-9]%', REVERSE(F.City))) AS FacilityCityName,\r\nSUBSTRING(F.State,1,2) as FacilityState,\r\nF.Zip as FacilityZip,\r\nF.FacilityGroup1,\r\nF.FacilityGroup2,\r\nF.AddressLine1 as FacilityAddressLine1, PR.ProviderCategory as ProviderName,\r\nSP.SpecialityCategory as Speciality,\r\nPTx.ProviderTaxonomyCategory as ProviderTaxonomyName,\r\nCASE WHEN py.PayerCategory IS NOT NULL THEN py.PayerCategory ELSE 'NOT AVAILABLE' END as PayerCategory,\r\nCASE WHEN py.PayerGroup IS NOT NULL THEN py.PayerGroup ELSE 'NOT AVAILABLE' END as PayerGroup,\r\nCASE WHEN Py.PayerGroup Like '%Medicare%' OR Py.PayerGroup Like '%Medicaid%' OR Py.PayerGroup Like '%Government%' THEN 'Federal'\r\nWHEN Py.PayerGroup Like '%Commercial%' THEN 'All Commercial'\r\nWHEN Py.PayerGroup Like '%Self%' or Py.PayerGroup Like '%No Ins%' THEN 'No Insurance/Self Pay'\r\nELSE 'Others' END AS InsurenceCategory,\r\nCASE WHEN (\r\nAppointmentStatusCategory = 'Cancel'\r\nOR DeleteFlag = 'Y'\r\n)\r\nAnd RescheduleFlag = 'N' THEN 0 WHEN (\r\nAppointmentStatusCategory = 'Expected'\r\nAND RescheduleFlag = 'N'\r\nAND DeleteFlag = 'N'\r\n) THEN 0 ELSE 1 END AS Label_NoKeep, isnull(\r\n(\r\nSELECT\r\nCOUNT(*) AS KeptCount\r\nFROM\r\nAPPOINTMENT A1\r\nLEFT OUTER JOIN APPOINTMENTSTATUS S on S.AppointmentStatusID = A1.AppointmentStatusID\r\nWHERE\r\nAppointmentStatusCategory = 'Kept'\r\nAND AppointmentStatusCategory <> 'Cancel'\r\nAND RescheduleFlag = 'N'\r\nAND DeleteFlag = 'N'\r\nand a1.appointmentDate < a.appointmentDate\r\nand p.patientid = a1.patientid\r\n),\r\n0\r\n) KeptCount, isnull(\r\n(\r\nSELECT\r\nCOUNT(*) AS NokeepCount\r\nFROM\r\nAPPOINTMENT A1\r\nLEFT OUTER JOIN APPOINTMENTSTATUS S on S.AppointmentStatusID = A.AppointmentStatusID\r\nWHERE\r\na1.appointmentDate < a.appointmentDate\r\nand p.patientid = a1.patientid\r\nand (\r\n(\r\nAppointmentStatusCategory = 'Expected'\r\nAND RescheduleFlag = 'N'\r\nAND DeleteFlag = 'N'\r\nAND A.AppointmentDate < Getdate()\r\n)\r\nOR (\r\n(\r\ns.AppointmentStatusCategory = 'Cancel'\r\nOR DeleteFlag = 'Y'\r\n)\r\nAnd RescheduleFlag = 'N'\r\n)\r\n)\r\n),\r\n0\r\n) NokeepCount, isnull(\r\n(\r\nSELECT\r\nCOUNT(*) AS Reschedule\r\nfrom\r\nAppointment a1\r\nwhere\r\na1.RescheduleFlag = 'y'\r\nand a1.AppointmentDate < a.AppointmentDate\r\nand a1.PatientID = p.PatientID\r\n),\r\n0\r\n) Reschedule,\r\nisnull(\r\n(\r\nSELECT\r\nCOUNT(*)\r\nfrom\r\nAppointment a1\r\nwhere\r\na1.AppointmentDate < a.AppointmentDate\r\nand a1.PatientID = p.PatientID\r\nand AppointmentStatusCategory = 'Kept'\r\nAND RescheduleFlag = 'N'\r\nAND DeleteFlag = 'N'\r\nand PatientID is not null\r\n),\r\n0\r\n) apptrank, isnull(\r\n(\r\nSELECT\r\ntop 1 datediff(\r\nd, a1.AppointmentDate, a.AppointmentDate\r\n)\r\nfrom\r\nAppointment a1\r\nwhere\r\na1.AppointmentDate < a.AppointmentDate\r\nand a1.PatientID = p.PatientID\r\nand AppointmentStatusCategory = 'Kept'\r\nAND RescheduleFlag = 'N'\r\nAND DeleteFlag = 'N'\r\nand PatientID is not null\r\norder by\r\na1.AppointmentDate desc\r\n),\r\n0\r\n) daysbetweenappt,\r\nisnull (\r\n(\r\nSELECT\r\ntop 1 appointmenttypecategory\r\nfrom\r\nAppointment a1\r\nleft outer join AppointmentType at on a1.AppointmentTypeID = at.AppointmentTypeID\r\nwhere\r\na1.AppointmentDate < a.AppointmentDate\r\nand a1.PatientID = p.PatientID\r\nand AppointmentStatusCategory = 'Kept'\r\nAND RescheduleFlag = 'N'\r\nAND DeleteFlag = 'N'\r\nand PatientID is not null\r\norder by\r\na1.AppointmentDate desc\r\n),\r\n'NONE'\r\n) prevappttype ,CASE WHEN FLOOR(DATEDIFF(DAY, p.DateOfBirth, GETDATE()) / 365) >= 0 and FLOOR(DATEDIFF(DAY, p.DateOfBirth, GETDATE()) / 365) <=50 THEN '<=50'\r\nWHEN FLOOR(DATEDIFF(DAY, p.DateOfBirth, GETDATE()) / 365) > 50 and FLOOR(DATEDIFF(DAY, p.DateOfBirth, GETDATE()) / 365) <=70 THEN '51-70'\r\nWHEN FLOOR(DATEDIFF(DAY, p.DateOfBirth, GETDATE()) / 365) > 70 and FLOOR(DATEDIFF(DAY, p.DateOfBirth, GETDATE()) / 365) <=90 THEN '71-90'\r\nWHEN FLOOR(DATEDIFF(DAY, p.DateOfBirth, GETDATE()) / 365) > 90 THEN '90+' END AS Age_Bins, CASE WHEN py.PayerCategory ='Non Contracted' THEN 1 ELSE 0 END as IsPrimaryNonContracted,\r\nCASE WHEN py.PayerCategory ='Self Pay' or py.PayerCategory ='NULL' THEN 1 ELSE 0 END as IsPrimarySelfPay,isnull(\r\n(\r\nSELECT\r\nCASE WHEN COUNT(*) = 0 THEN '0'\r\nWHEN COUNT(*) >0 and COUNT(*) <=5 THEN '1-5'\r\nWHEN COUNT(*) >5 and COUNT(*) <=10 THEN '6-10'\r\nWHEN COUNT(*) >10 THEN '>10' END as Bins_apptrank\r\nfrom\r\nAppointment a1\r\nleft outer join AppointmentType at on a1.AppointmentTypeID = at.AppointmentTypeID\r\nwhere\r\na1.AppointmentDate < a.AppointmentDate\r\nand a1.PatientID = p.PatientID\r\nand AppointmentStatusCategory = 'Kept'\r\nAND RescheduleFlag = 'N'\r\nAND DeleteFlag = 'N'\r\nand PatientID is not null\r\n),\r\n'NONE'\r\n) Bins_apptrank\r\nFROM\r\nAPPOINTMENT A\r\nINNER JOIN APPOINTMENTSTATUS S on S.AppointmentStatusID = A.AppointmentStatusID\r\nINNER JOIN AppointmentStates APS on APS.AppointmentID = A.AppointmentID\r\nINNER JOIN APPOINTMENTTYPE AT ON AT.AppointmentTypeID = A.AppointmentTypeID\r\nLeft outer JOIN PATIENT P ON P.PatientID = A.PatientID\r\nINNER JOIN PATIENTPAYER PP ON PP.PatientId = P.PatientId\r\nLeft outer JOIN (select distinct raceid, Race from Race ) r ON r.RaceID = P.RaceID\r\nLeft outer JOIN ZipCodeInfo ZC ON ZC.zipcode = SUBSTRING(P.Zip,1,5)\r\nLEFT OUTER JOIN PROVIDER PR ON PR.ProviderID = A.ProviderID\r\nLEFT OUTER JOIN ProviderTaxonomy PTx ON PR.TaxonomyID = PTx.ProviderTaxonomyID\r\nLEFT OUTER JOIN ReferringProvider RP ON RP.ReferringProviderID = A.ReferringProviderID\r\nLEFT OUTER JOIN SPECIALITY SP ON SP.SpecialityID = PR.SpecialityID\r\nLEFT OUTER JOIN FACILITY F ON F.Facilityid = A.FacilityID\r\nLEFT OUTER JOIN PRACTICE PRC ON PRC.PracticeID = A.PracticeID\r\nLEFT OUTER JOIN PAYER PY ON PY.PayerID = PP.PayerID\r\nWHERE\r\na.AppointmentDate between '01/01/2021' AND '11/30/2021'\r\nand p.PatientID is not null AND PP.DefCob = 1 and pp.PatientPayerID IS NOT NULL", 'dbtype': 'mssql', 'is_header': 'Use Header Line', 'server_url': '/numtraPlatform/NumtraPlatformV2/uploads/platform/'}")
	#sourcePostExecutionHook(sqlserver)

except Exception as ex: 
	logging.error(ex)


***TRANSFORMING DATAFRAME***

In [ ]:
#%run wfanoshowcanceldistanceinfoxgbdHooks.ipynb
try:
	#transformationPreExecutionHook()

	autofe = TransformationMain.run(sqlserver,json.dumps( {"FE": [{"transformationsData": [{"feature_label": "policynumber", "transformation_label": "String Indexer"}], "feature": "policynumber", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "6896866993.08", "stddev": "27954276235.5", "min": "", "max": "21551825", "missing": "0", "distinct": "402"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "policynumber"}, {"transformationsData": [{"feature_label": "AppointmentID", "transformation_label": "String Indexer"}], "feature": "AppointmentID", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "0004A03B-ECD3-40EF-A3D9-AA1875A6752D", "max": "FFE29399-288E-4F85-B66D-3D7227A0AED9", "missing": "0", "distinct": "500"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "AppointmentID"}, {"transformationsData": [{"feature_label": "AppointmentDate", "transformation_label": "Extract Date"}], "feature": "AppointmentDate", "type": "date", "selected": "True", "replaceby": "random", "stats": {"count": "", "mean": "", "stddev": "", "min": "", "max": "", "missing": "0"}, "transformation": [{"transformation": "Extract Date", "selectedAsDefault": 1}], "generated": "False", "updatedLabel": "AppointmentDate"}, {"transformationsData": [{"feature_label": "PatientID", "transformation_label": "String Indexer"}], "feature": "PatientID", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "017E3AD3-7931-499D-8CEB-C7F5FCAF574C", "max": "FF35B960-B0D8-49A2-A9B0-3190583199EA", "missing": "0", "distinct": "411"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "PatientID"}, {"transformationsData": [{"feature_label": "PatientNumber", "transformation_label": "String Indexer"}], "feature": "PatientNumber", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "1210517.23", "stddev": "849566.96", "min": "100530", "max": "99834", "missing": "0", "distinct": "411"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "PatientNumber"}, {"transformationsData": [{"feature_label": "Gender", "transformation_label": "String Indexer"}], "feature": "Gender", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "F", "max": "M", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Gender"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "PatientAge", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "66.22", "stddev": "12.37", "min": "24", "max": "96", "missing": "0"}, "updatedLabel": "PatientAge"}, {"transformationsData": [{"feature_label": "PatientCity", "transformation_label": "String Indexer"}], "feature": "PatientCity", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Atlanta", "max": "Wheeling", "missing": "0", "distinct": "37"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "PatientCity"}, {"transformationsData": [{"feature_label": "PatientZipcode", "transformation_label": "String Indexer"}], "feature": "PatientZipcode", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "43400.92", "stddev": "16941.32", "min": "08807", "max": "95899", "missing": "0", "distinct": "46"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "PatientZipcode"}, {"transformationsData": [{"feature_label": "PatientEthnicityID", "transformation_label": "String Indexer"}], "feature": "PatientEthnicityID", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "1bc90f3d-2442-410d-8814-ccc7e81da7a1", "max": "cbd0af81-70ad-4e34-afa6-0a5f10ac035c", "missing": "0", "distinct": "7"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "PatientEthnicityID"}, {"transformationsData": [{"feature_label": "PatientRace", "transformation_label": "String Indexer"}], "feature": "PatientRace", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Asian", "max": "White", "missing": "0", "distinct": "6"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "PatientRace"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Distance", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "424.79", "stddev": "402.35", "min": "0.0", "max": "2054.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Distance"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "population", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "2473.38", "stddev": "7739.3", "min": "0", "max": "51469", "missing": "0"}, "updatedLabel": "population"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "mean_income", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "10724.12", "stddev": "32101.22", "min": "0", "max": "132717", "missing": "0"}, "updatedLabel": "mean_income"}, {"transformationsData": [{"feature_label": "PatientType", "transformation_label": "String Indexer"}], "feature": "PatientType", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "EstPt", "max": "NewPt", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "PatientType"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "days_AppDate_AppCreatDate", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "84.84", "stddev": "106.13", "min": "-3", "max": "392", "missing": "0"}, "updatedLabel": "days_AppDate_AppCreatDate"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "CreatedBy", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1519.82", "stddev": "1003.3", "min": "-782", "max": "3131", "missing": "0"}, "updatedLabel": "CreatedBy"}, {"transformationsData": [{"feature_label": "BinsAppointmentDuration", "transformation_label": "String Indexer"}], "feature": "BinsAppointmentDuration", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "1-3 hrs", "max": "Lt 10 Mins", "missing": "0", "distinct": "4"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "BinsAppointmentDuration"}, {"transformationsData": [{"feature_label": "TimeOfAppointment", "transformation_label": "String Indexer"}], "feature": "TimeOfAppointment", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "AM", "max": "PM", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "TimeOfAppointment"}, {"transformationsData": [{"feature_label": "BinsTimeOfAppointment", "transformation_label": "String Indexer"}], "feature": "BinsTimeOfAppointment", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Early Morning", "max": "Post Lunch/Evening", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "BinsTimeOfAppointment"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "DayoftheWeek", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "3.89", "stddev": "1.41", "min": "2", "max": "7", "missing": "0"}, "updatedLabel": "DayoftheWeek"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "MonthoftheYear", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "4.58", "stddev": "2.59", "min": "1", "max": "11", "missing": "0"}, "updatedLabel": "MonthoftheYear"}, {"transformationsData": [{"feature_label": "AppointmentType", "transformation_label": "String Indexer"}], "feature": "AppointmentType", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "1 - Cataract Consult", "max": "Z - YAG", "missing": "0", "distinct": "73"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "AppointmentType"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "EligibilityFlag", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.0", "stddev": "0.0", "min": "0", "max": "0", "missing": "0"}, "updatedLabel": "EligibilityFlag"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "ConfirmationFlag", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.04", "stddev": "0.2", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "ConfirmationFlag"}, {"transformationsData": [{"feature_label": "PracticeName", "transformation_label": "String Indexer"}], "feature": "PracticeName", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "PracticeName1", "max": "PracticeName2", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "PracticeName"}, {"transformationsData": [{"feature_label": "FacilityCategory", "transformation_label": "String Indexer"}], "feature": "FacilityCategory", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Location107", "max": "Location96", "missing": "0", "distinct": "39"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "FacilityCategory"}, {"transformationsData": [{"feature_label": "FacilityCityName", "transformation_label": "String Indexer"}], "feature": "FacilityCityName", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Batavia", "max": "Troy", "missing": "0", "distinct": "19"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "FacilityCityName"}, {"transformationsData": [{"feature_label": "FacilityState", "transformation_label": "String Indexer"}], "feature": "FacilityState", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "IN", "max": "OH", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "FacilityState"}, {"transformationsData": [{"feature_label": "FacilityZip", "transformation_label": "String Indexer"}], "feature": "FacilityZip", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "449621676.45", "stddev": "11066586.44", "min": "410173415", "max": "470251055", "missing": "0", "distinct": "33"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "FacilityZip"}, {"transformationsData": [{"feature_label": "FacilityGroup1", "transformation_label": "String Indexer"}], "feature": "FacilityGroup1", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "ASC", "max": "VEI Office", "missing": "0", "distinct": "11"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "FacilityGroup1"}, {"transformationsData": [{"feature_label": "FacilityGroup2", "transformation_label": "String Indexer"}], "feature": "FacilityGroup2", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Indiana", "max": "Ohio", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "FacilityGroup2"}, {"transformationsData": [{"feature_label": "FacilityAddressLine1", "transformation_label": "String Indexer"}], "feature": "FacilityAddressLine1", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "10615 Montgomery Rd., Suite 202", "max": "89 Sylvania Drive", "missing": "0", "distinct": "36"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "FacilityAddressLine1"}, {"transformationsData": [{"feature_label": "ProviderName", "transformation_label": "String Indexer"}], "feature": "ProviderName", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Provider10749", "max": "Provider9994", "missing": "0", "distinct": "117"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "ProviderName"}, {"transformationsData": [{"feature_label": "Speciality", "transformation_label": "String Indexer"}], "feature": "Speciality", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "SpecialityCategory-11_Demo3", "max": "SpecialityCategory6465_Demo3", "missing": "0", "distinct": "11"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Speciality"}, {"transformationsData": [{"feature_label": "ProviderTaxonomyName", "transformation_label": "String Indexer"}], "feature": "ProviderTaxonomyName", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Allopathic & Osteopathic Physicians : Ophthalmology", "max": "Unspecified", "missing": "0", "distinct": "7"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "ProviderTaxonomyName"}, {"transformationsData": [{"feature_label": "PayerCategory", "transformation_label": "String Indexer"}], "feature": "PayerCategory", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Aetna Commercial", "max": "United Commercial", "missing": "0", "distinct": "16"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "PayerCategory"}, {"transformationsData": [{"feature_label": "PayerGroup", "transformation_label": "String Indexer"}], "feature": "PayerGroup", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Aetna Commercial", "max": "United Commercial", "missing": "0", "distinct": "16"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "PayerGroup"}, {"transformationsData": [{"feature_label": "InsurenceCategory", "transformation_label": "String Indexer"}], "feature": "InsurenceCategory", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "All Commercial", "max": "Others", "missing": "0", "distinct": "4"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "InsurenceCategory"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Label_NoKeep", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.61", "stddev": "0.49", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "Label_NoKeep"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "KeptCount", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "27.17", "stddev": "30.74", "min": "0", "max": "280", "missing": "0"}, "updatedLabel": "KeptCount"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "NokeepCount", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "17.09", "stddev": "36.0", "min": "0", "max": "518", "missing": "0"}, "updatedLabel": "NokeepCount"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Reschedule", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "7.23", "stddev": "10.83", "min": "0", "max": "106", "missing": "0"}, "updatedLabel": "Reschedule"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "apptrank", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "16.83", "stddev": "31.82", "min": "0", "max": "323", "missing": "0"}, "updatedLabel": "apptrank"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "daysbetweenappt", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "68.57", "stddev": "289.48", "min": "0", "max": "3268", "missing": "0"}, "updatedLabel": "daysbetweenappt"}, {"transformationsData": [{"feature_label": "prevappttype", "transformation_label": "String Indexer"}], "feature": "prevappttype", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "1 - Cataract Consult", "max": "Z - Surgery (Plastics)", "missing": "0", "distinct": "56"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "prevappttype"}, {"transformationsData": [{"feature_label": "Age_Bins", "transformation_label": "String Indexer"}], "feature": "Age_Bins", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "51-70", "max": "<=50", "missing": "0", "distinct": "4"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Age_Bins"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "IsPrimaryNonContracted", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.0", "stddev": "0.0", "min": "0", "max": "0", "missing": "0"}, "updatedLabel": "IsPrimaryNonContracted"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "IsPrimarySelfPay", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.02", "stddev": "0.13", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "IsPrimarySelfPay"}, {"transformationsData": [{"feature_label": "Bins_apptrank", "transformation_label": "String Indexer"}], "feature": "Bins_apptrank", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "0.0", "stddev": "0.0", "min": "0", "max": ">10", "missing": "0", "distinct": "4"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Bins_apptrank"}]}))

	#transformationPostExecutionHook(autofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run wfanoshowcanceldistanceinfoxgbdHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=XG-Boost-classifier(autofe, ["PatientAge", "Distance", "population", "mean_income", "days_AppDate_AppCreatDate", "CreatedBy", "DayoftheWeek", "MonthoftheYear", "EligibilityFlag", "ConfirmationFlag", "KeptCount", "NokeepCount", "Reschedule", "apptrank", "daysbetweenappt", "IsPrimaryNonContracted", "IsPrimarySelfPay", "policynumber_stringindexer", "AppointmentID_stringindexer", "AppointmentDate_dayofmonth", "AppointmentDate_month", "AppointmentDate_year", "PatientID_stringindexer", "PatientNumber_stringindexer", "Gender_stringindexer", "PatientCity_stringindexer", "PatientZipcode_stringindexer", "PatientEthnicityID_stringindexer", "PatientRace_stringindexer", "PatientType_stringindexer", "BinsAppointmentDuration_stringindexer", "TimeOfAppointment_stringindexer", "BinsTimeOfAppointment_stringindexer", "AppointmentType_stringindexer", "PracticeName_stringindexer", "FacilityCategory_stringindexer", "FacilityCityName_stringindexer", "FacilityState_stringindexer", "FacilityZip_stringindexer", "FacilityGroup1_stringindexer", "FacilityGroup2_stringindexer", "FacilityAddressLine1_stringindexer", "ProviderName_stringindexer", "Speciality_stringindexer", "ProviderTaxonomyName_stringindexer", "PayerCategory_stringindexer", "PayerGroup_stringindexer", "InsurenceCategory_stringindexer", "prevappttype_stringindexer", "Age_Bins_stringindexer", "Bins_apptrank_stringindexer"], "Label_NoKeep")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)
